<div class="alert alert-block alert-danger">
<b>Check the Kernel you are using:</b> Before we get started, if you are running this on HiPerGator, double check the kernel in use. This is shown in the top right of the window and should look like: <img src="images/kernel.python310.png" alt"Image showing that the notebook is using the Python 3.10 Full kernel" style="float:right">
</div>

# Introduction to NumPy

This notebook is based on [chapter 2 of Jake VanderPlas' Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/02.00-introduction-to-numpy.html). [<img src="images/PDSH-cover-small.png" alt="PDSH Cover Image" style="width: 50px;float:right"/>](https://jakevdp.github.io/PythonDataScienceHandbook/02.00-introduction-to-numpy.html)

> This chapter, along with [chapter 3](https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html), outlines techniques for effectively loading, storing, and manipulating in-memory data in Python. The topic is very broad: datasets can come from a wide range of sources and a wide range of formats, including be collections of documents, collections of images, collections of sound clips, collections of numerical measurements, or nearly anything else. **Despite this apparent heterogeneity, it will help us to think of all data fundamentally as arrays of numbers.**

> For example, images–particularly **digital images**–can be thought of as simply two-dimensional arrays of numbers representing pixel brightness across the area. **Sound clips** can be thought of as one-dimensional arrays of intensity versus time. **Text** can be converted in various ways into numerical representations, perhaps binary digits representing the frequency of certain words or pairs of words. **No matter what the data are, the first step in making it analyzable will be to transform them into arrays of numbers.** (We will discuss some specific examples of this process later in [Feature Engineering](https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html))

> For this reason, efficient storage and manipulation of numerical arrays is absolutely fundamental to the process of doing data science. We'll now take a look at the specialized tools that Python has for handling such numerical arrays: the NumPy package, and the Pandas package (discussed in Chapter 3).

> This chapter will cover NumPy in detail. NumPy (short for **Numerical Python**) provides an efficient interface to store and operate on dense data buffers. In some ways, NumPy arrays are like Python's built-in list type, but **NumPy arrays provide much more efficient storage and data operations as the arrays grow larger in size.** NumPy arrays form the core of nearly the entire ecosystem of data science tools in Python, so time spent learning to use NumPy effectively will be valuable no matter what aspect of data science interests you.



Now we can import and look at the version of NumPy:

In [2]:
import numpy as np

np.__version__

'2.2.5'

<div class="alert alert-block alert-info">
    <b>Note:</b> the __ methods of functions, like <code>__version__</code> are referred to as the "double underscore" or "dunder" methods and are generally not intended to be directly interacted with by users.
</div>

Remember the built in documentation with `<TAB>` and `?`.

In [4]:
np?

Type:        module
String form: <module 'numpy' from '/home/codespace/.local/lib/python3.12/site-packages/numpy/__init__.py'>
File:        ~/.local/lib/python3.12/site-packages/numpy/__init__.py
Docstring:  
NumPy
=====

Provides
  1. An array object of arbitrary homogeneous items
  2. Fast mathematical operations over arrays
  3. Linear Algebra, Fourier Transforms, Random Number Generation

How to use the documentation
----------------------------
Documentation is available in two forms: docstrings provided
with the code, and a loose standing reference guide, available from
`the NumPy homepage <https://numpy.org>`_.

We recommend exploring the docstrings using
`IPython <https://ipython.org>`_, an advanced Python shell with
TAB-completion and introspection capabilities.  See below for further
instructions.

The docstring examples assume that `numpy` has been imported as ``np``::

  >>> import numpy as np

Code snippets are indicated by three greater-than signs::

  >>> x = 42
  >>> x 

## Understanding Data Types in Python

As we've mentioned earlier, Python is **dynamically typed**. This flexibility can be handy, but, especially as sizes of datasets grow, if becomes a liability.

As the PDSH chapter points out, languages such as C and Java are **statically typed**, meaning that the programmer has to declare, when the variable is created, the type of data that the variable will store.

For example in C, you could make a loop like this:

```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

While in Python the same thing is done like this:

```Python
# Python code
result = 0
for i in range(100):
    result += i
```

Notice that in C, the data types, all `int`s in the example, is explicitly declared, while in Python it is dynamically inferred.

## A Python Integer is More Than Just and Integer

We also briefly saw in the Intro to Jupyter session that a lot of the underlying code for Python is actually written in C (that is why the `??` function can't always display the code for a function). 

In the Intro to Python session, we learned about Object Oriented Programming, and that everything in Python is an object--even strings and integers.

All of this leads to the reality that if we do something like `x=1000`, `x` is not just a "raw" integer--bits stored in memory. 

> It's actually a pointer to a compound C structure, which contains several values. Looking through the Python 3.4 source code, we find that the integer (long) type definition effectively looks like this (once the C macros are expanded):

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```
> A single integer in Python 3.4 actually contains four pieces:
> * `ob_refcnt`, a reference count that helps Python silently handle memory allocation and deallocation
> * `ob_type`, which encodes the type of the variable
> * `ob_size`, which specifies the size of the following data members
ob_digit, which contains the actual integer value that we expect the Python variable to represent.

> This means that there is some overhead in storing an integer in Python as compared to an integer in a compiled language like C, as illustrated in the following figure:

<figure>
  <img src="images/cint_vs_pyint.png" alt="Memory storage for C vs Python integers from Python Data Science Handbook">
  <figcaption>Memory storage for C vs Python integers, from Python Data Science Handbook</figcaption>
</figure>

> Here `PyObject_HEAD` is the part of the structure containing the reference count, type code, and other pieces mentioned before.

> Notice the difference here: **a C integer is essentially a label for a position in memory whose bytes encode an integer value**. **A Python integer is a pointer to a position in memory containing all the Python object information**, including the bytes that contain the integer value. This extra information in the Python integer structure is what allows Python to be coded so freely and dynamically. All this additional information in Python types comes at a cost, however, which becomes especially apparent in structures that combine many of these objects.

## A Python List is More Than Just a List

Practice creating some lists, and remember that a list can contain one or more data types--i.e. a Python list can have heterogenous data types.

In [ ]:
# Create some lists
list_a = [1,2,3.0,'four']



As PDSH notes, this sets up the situation where each element of a list needs to store its own information about the element's data type:

<figure>
  <img src="images/PDSH_list.png" alt="Python list image from Python Data Science Handbook">
  <figcaption>Memory storage for a Python list, from Python Data Science Handbook</figcaption>
</figure>

As you can imagine, this becomes exceedingly inefficient if, for example you have a list of 1,000,000 integers. 

## Creating Array from Python Lists

PDSH mentions that there is a `array` module, but I rarely see anyone using it, so let's skip to the NumPy [`ndarry`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html)--a multi-dimensional array. NumPy not only provides the data structure, but also highly efficient operations on the data.

First, we can create a NumPy array from a Python list:

In [6]:
# Integer array from list
np.array([1,2,3,4])


array([1, 2, 3, 4])

NumPy arrays are constrained such that **all** elements need to be of the same type. 

If possible, NumPy will *upcast* items to create an array of matching type.

In [12]:
# Create an array with a mix of integers and floats
np.array([1,2,3,4.1])




array([1. , 2. , 3. , 4.1])

In [11]:
# What about strings?

np.array([1,2,3.0,'four'])

array(['1', '2', '3.0', 'four'], dtype='<U32')

In [16]:
# You can also specify the type if you want

np.array([1,2,3,4], dtype=np.float16)


array([1., 2., 3., 4.], dtype=float16)

And, as the `ndarray` name implies, arrays can be multidimensional.

In [ ]:
np.array([range(i,1+3) for i in [2,4,6]])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

## Creating Arrays from Scratch

> Especially for larger arrays, it is more efficient to create arrays from scratch using routines built into NumPy. Here are several examples:

In [19]:
# Create a length-10 integer array filled with zeros
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
# Create a 3x5 floating-point array filled with ones
np.ones((3,5), dtype=float)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [21]:
# Create an array filled with a linear sequence
# Starting at 0, ending at 20, stepping by 2
# (this is similar to the built-in range() function)   

np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [23]:
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [24]:
# Create a 3x3 array of uniformly distributed random values between 0 and 1
np.random.random((3,3))

array([[0.01359513, 0.68077782, 0.60492897],
       [0.50919687, 0.70932565, 0.37198979],
       [0.24264934, 0.44080862, 0.30695209]])

In [26]:
# Create a 3x3 array of normally distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3,3))

array([[-0.0951582 , -0.95464008,  0.72140531],
       [ 1.35387004,  0.45983904, -0.09012835],
       [ 1.4986161 , -0.11667986, -0.37410418]])

In [27]:
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3,3))

array([[4, 8, 2],
       [2, 7, 2],
       [4, 3, 8]])

In [28]:
# Create a 3x3 identity matrix
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [29]:
# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location

np.empty(3, dtype=int)

array([4607182418800017408, 4607182418800017408, 4607182418800017408])

### Exercise 1

Create the following types of NumPy arrays.
 * A 4X4 matrix with ones in every cell
 * A 6X6 matrix with ones on the diagonal from top left to bottom right.
 * A 3X3X3 matrix with normally distributed random numbers with mean of 5 and standard deviation of 2
 * A vector with 1,000,000 evenly spaced numbers between five and 10.

In [ ]:
# Your code here
#a 4x4 matrix with one in every cell
np.ones((4,4))
#a 6x6 matrix with ones in the diagonal from top left to bottom right
np.eye(6)
#a3x3x3 matrix with normally distributed random numbers with mean of 5 and standard deviation of 2
np.random.normal(5, 2, (3,3,3))
#a vector with 1000000 evenly spaced numbers between five and ten
np.linspace(5, 10, 1000000)

[ 5.        5.000005  5.00001  ...  9.99999   9.999995 10.      ]


In [36]:
# %load snippets/NumPy_Ex_01.matrices.py
# A 4X4 matrix with ones in every cell
ones = np.ones((4, 4))
print(ones, "\n\n")

# A 6X6 matrix with ones on the diagonal from top left to bottom right.
diag = np.eye(6)
print(diag, "\n\n")

# A 3X3X3 matrix with normally distributed random numbers with mean of 5 and standard deciavion of 2
cube = np.random.normal(5, 2, (3, 3, 3))
print(cube, "\n\n")

# A vector with 1,000,000 evenly spaced numbers between five and 10.
np.linspace(5, 10, 1000000)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]] 


[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]] 


[[[6.16576219 7.03994884 6.97973544]
  [4.71842436 5.67150855 4.46046086]
  [3.47033534 5.10730529 6.73907156]]

 [[4.04363163 5.85710363 4.65124305]
  [7.74079115 4.38617288 8.6385793 ]
  [2.21007876 5.12910473 4.02986424]]

 [[5.2490445  4.53982159 3.91672756]
  [7.29376533 3.91438387 4.65387931]
  [3.76143229 3.25529798 0.78004229]]] 




array([ 5.      ,  5.000005,  5.00001 , ...,  9.99999 ,  9.999995,
       10.      ], shape=(1000000,))

## NumPy Standard Data Types

> NumPy arrays contain values of a single type, so it is important to have detailed knowledge of those types and their limitations. Because NumPy is built in C, the types will be familiar to users of C, Fortran, and other related languages.

Notice that there are **a lot** of data types, and there are even more options if needed. This is one reason Python is handy. But again, that flexibility comes at a cost. 

Data type | Description
----------|------------
bool_ | Boolean (True or False) stored as a byte
int_ | Default integer type (same as C long; normally either int64 or int32)
intc | Identical to C int (normally int32 or int64)
intp | Integer used for indexing (same as C size_t; normally either int32 or int64)
int8 | Byte (-128 to 127)
int16 | Integer (-32768 to 32767)
int32 | Integer (-2147483648 to 2147483647)
int64 | Integer (-9223372036854775808 to 9223372036854775807)
uint8 | Unsigned integer (0 to 255)
uint16 | Unsigned integer (0 to 65535)
uint32 | Unsigned integer (0 to 4294967295)
uint64 | Unsigned integer (0 to 18446744073709551615)
float_ | Shorthand for float64.
float16 | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa
float32 | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa
float64 | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa
complex_ | Shorthand for complex128.
complex64 | Complex number, represented by two 32-bit floats
complex128 | Complex number, represented by two 64-bit floats

## The bascis of NumPy Arrays

> Data manipulation in Python is nearly synonymous with NumPy array manipulation: even newer tools like Pandas ([Chapter 3](https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html)) are built around the NumPy array. Though note that as datasets grow, tools like Nvidia's [RAPIDS](https://rapids.ai/) framework are replacing Pandas by moving calculations onto the GPU, accelerating calculations. But, fear not, the Pandas and RAPIDS are largely compatible.


### A note on random numbers

PDSH moves on to creating three arrays to use for the following examples. Before we get there, let's take a look at the first thing that is done:

`np.random.seed(0)`

This sets the random number generator seed to 0. What does that mean?? Well, computers really can't make truly random numbers. What they use is a complex series of manipulations to generate numbers that appear random, sometimes called *pseudorandom*. If you start with the same number, the seed, the sequence of "random" numbers generated is **guaranteed** to be identical. This has good and bad properties. On the good side, we can set a seed and all have the same numbers, you can also use this for troubleshooting, etc. On the bad side, we are often lulled into a false sense of having simulated something repeatedly only to find that we failed to consider the biases that may be introduced by the random number generator--or worse, repeatedly simulating something using the same seed!

Also, as a note, this guarantee only applies to identical code. PDSH used NumPy version 1.11.1, while we (using Python 3.8 full kernel on HiPerGator on 1/17/21) are using 1.13.1--while we will get consistent numbers from run to run and student to student, our numbers are different than in the text.

### Create some arrays to use

In [ ]:
# Create some sample arrays

  # Set random number generator seed for reproducibility
np.random.seed(0)
  # One-dimensional array
x1 = np.random.randint(10, size=6)
  # Two-dimensional array
x2 = np.random.randint(10, size=(3, 4))
  # Three-dimensional array
x3 = np.random.randint(10, size=(2, 3, 4))


[[[8 1 5 9]
  [8 9 4 3]
  [0 3 5 0]]

 [[2 3 8 1]
  [3 3 3 7]
  [0 1 9 9]]]


### NumPy Array Attributes:

> Each array has attributes `ndim` (the number of dimensions), `shape` (the size of each dimension), and `size` (the total size of the array):

In [39]:
print("x1 ndim: ", x1.ndim)
print("x2 shape:", x2.shape)
print("x3 size: ", x3.size)

x1 ndim:  1
x2 shape: (3, 4)
x3 size:  24


In [41]:
# Print the data type of the array
print(x2.dtype)

int64


In [42]:
# Print the itemsize and nbytes
print("itemsize:", x2.itemsize, "bytes")
print("nbytes:", x2.nbytes, "bytes")

itemsize: 8 bytes
nbytes: 96 bytes


### Array Indexing: Accessing Single Elements

This is similar to using lists in Python.

In [44]:
x1

array([5, 0, 3, 3, 7, 9])

In [45]:
x1[0]

np.int64(5)

In [46]:
# Indexing from the end of the array
x1[-1]

np.int64(9)

In multi-dimensional arrays, items are accessed using a comma-separated list of indices:

In [48]:
x2[0, 0]

np.int64(3)

### Array Slicing: Accessing Subarrays

As with lists, NumPy array use slices.

#### Slices for one-dimensional arrays

In [ ]:
 # first 5 elements

In [ ]:
 # elements from index 5 on

In [ ]:
 # middle sub-array

In [ ]:
 # every other element

In [ ]:
 # every other element, starting at index 1

> A potentially confusing case is when the `step` value is negative. In this case, the defaults for `start` and `stop` are swapped. This becomes a convenient way to reverse an array:

In [ ]:
  # all elements, reversed

In [ ]:
  # reversed every other from index 5

#### Slices for multi-dimensional subarrays

> Multi-dimensional slices work in the same way, with multiple slices separated by commas. For example:

In [ ]:
  # two rows, three columns

In [ ]:
 # all rows, every other column

In [ ]:
 # first column of x2

In [ ]:
  # first row of x2

In [ ]:
 # equivalent to x2[0, :]

## Subarrays as no-copy views

An important--and at times both useful and confusing--thing to know about array slices is that they return *views* rather than *copies* of the array data. Changing data in a subarray, changes the data in the originating array.

In [ ]:
# Extract a 2X2 subarray from this



In [ ]:
# Modify element of x2_sub


> This default behavior is actually quite useful: it means that when we work with large datasets, we can access and process pieces of these datasets without the need to copy the underlying data buffer.

#### Creating copies of arrays

If what you want is really a copy, you can use the `.copy()` method.

### Reshaping Arrays

Another common action is to reshape the dimensions of an array. The `.reshape()` method is the easiest way to do this. 

In [ ]:
# Create a row array



In [ ]:
# Reshape to column vector using newaxis


## Concatenation of arrays 

There are several functions to concatenate two arrays in NumPy: `np.concatenate`, `np.vstack`, and `np.hstack` are common methods

In [ ]:
# Concatenating multiple arrays


In [ ]:
# Concatenating 2-dimensional arrays



In [ ]:
# Concatenating along the second axis (zero-indexed)



In [ ]:
# For mixed dimension arrays, vstack and hstack are more clear



In [ ]:
# Need to be careful of dimensions


### Splitting Arrays

In [ ]:
# Split after 3rd and 5th elements



# Computation on NumPy Arrays: Universal Functions

> Up until now, we have been discussing some of the basic nuts and bolts of NumPy; in the next few sections, we will dive into the reasons that NumPy is so important in the Python data science world. Namely, it provides an easy and flexible interface to optimized computation with arrays of data.

> Computation on NumPy arrays can be very fast, or it can be very slow. The key to making it fast is to use vectorized operations, generally implemented through NumPy's universal functions (ufuncs). This section motivates the need for NumPy's ufuncs, which can be used to make repeated calculations on array elements much more efficient. It then introduces many of the most common and useful arithmetic ufuncs available in the NumPy package.

## The Slowness of Loops

Both the dynamic typing and the interpreted nature of Python lead to slowness. PDSH talks about several options to circumvent some of this, and we will return to some throughout the semester.

One thing to keep in ming though is that: 

> The relative sluggishness of Python generally manifests itself in situations where many small operations are being repeated – for instance looping over arrays to operate on each element. For example, imagine we have an array of values and we'd like to compute the reciprocal of each. A straightforward approach might look like this:

In [ ]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

In [ ]:
# Let's time this on a big array:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

> It turns out that the bottleneck here is not the operations themselves, but the type-checking and function dispatches that CPython must do at each cycle of the loop. Each time the reciprocal is computed, Python first examines the object's type and does a dynamic lookup of the correct function to use for that type. If we were working in compiled code instead, this type specification would be known before the code executes and the result could be computed much more efficiently.

## Introducing UFuncs

NumPy provides a convenient interface into a statically types, compiled routine in a **vectorized** operation.

Let's compare the Python implementation to the UFunction that 

I think we can skip the rest of this section...there may be things we come back to, but I think this gets a bit into the weeds here.

## Aggregations: Min, Max, and Everything In Between

### Summing the Values in an Array

Again the main take home here is that NumPy, both through its compiled code and its explicit typing, speeds up calculations. For example, summing a NumPy array of 1,000,000 random numbers can be done with both the built-in `sum()` function and the `np.sum()` function, which is much faster:

### Minimum and Maximum

Similarly, the NumPy versions of these are faster:

> For min, max, sum, and several other NumPy aggregates, a shorter syntax is to use methods of the array object itself:

> Whenever possible, make sure that you are using the NumPy version of these aggregates when operating on NumPy arrays!

## Multidimensional aggregates

For N-dimensional matrices, you can aggregate along different axes:

In [ ]:
# E.g. a two-dimensional matrix



In [ ]:
# By default, the aggregation is over the entire array


In [ ]:
# You can specify the axis


> The way the axis is specified here can be confusing to users coming from other languages. The `axis` keyword specifies the *dimension of the array that will be collapsed*, rather than the dimension that will be returned. So specifying `axis=0` means that the first axis will be collapsed: for two-dimensional arrays, this means that values within each column will be aggregated.